In [6]:
df = df.head()

In [7]:
df

,text,source,text_length,word_count,Prompt,lexical_diversity
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,967,157,Undefined,0.806630
1,Miles feels restless after working all day. He...,Bloom-7B,5068,778,Undefined,0.661255
2,So first of I am danish. That means that I fol...,Bloom-7B,1602,267,Undefined,0.718354
3,In this paper we present a novel rule-based ap...,Bloom-7B,5469,848,Undefined,0.564532
4,"Most social progressives, love democracy, and ...",Bloom-7B,2379,380,Undefined,0.752860


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from textstat.textstat import textstatistics
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec, KeyedVectors
import nltk
import pickle

# Ensure nltk resources are downloaded (needed for tokenization)
nltk.download('punkt')

# Define feature extraction functions
def lexical_diversity(text):
    """Compute lexical diversity as the ratio of unique tokens to total tokens."""
    tokens = word_tokenize(text)
    return len(set(tokens)) / len(tokens) if tokens else 0

def average_sentence_length(text):
    """Compute the average sentence length."""
    sentences = sent_tokenize(text)
    tokens = word_tokenize(text)
    return len(tokens) / len(sentences) if sentences else 0

def compute_readability_score(text):
    """Compute a readability score (e.g., Flesch Reading Ease)."""
    return textstatistics().flesch_reading_ease(text)

def compute_text_length_metrics(text):
    """Compute various text length metrics."""
    chars = len(text)
    words = len(word_tokenize(text))
    sentences = len(sent_tokenize(text))
    return chars, words, sentences

def compute_prompt_features(prompt):
    """Compute features based on prompts, such as prompt length."""
    prompt_length = len(prompt)
    return prompt_length

def extract_nlp_embeddings(text, model):
    """Extract word embeddings for the given text and average them, compatible with Gensim 4.0.0 and later."""
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    
    # Check if each token is in the model's vocabulary using .key_to_index
    tokens_in_model = [word for word in tokens if word in model.key_to_index]
    
    # If no tokens are found in the model's vocabulary, return a zero vector
    if not tokens_in_model:
        return np.zeros(model.vector_size)
    
    # Compute the average embedding for the tokens found in the model's vocabulary
    embeddings = np.mean([model[word] for word in tokens_in_model], axis=0)
    return embeddings

def add_features_to_df(df, model):
    """Add computed features to the DataFrame."""
    df['lexical_diversity'] = df['text'].apply(lexical_diversity)
    df['avg_sentence_length'] = df['text'].apply(average_sentence_length)
    df['readability_score'] = df['text'].apply(compute_readability_score)
    
    # Adding text length metrics
    text_length_metrics = df['text'].apply(compute_text_length_metrics)
    df['char_count'], df['word_count'], df['sentence_count'] = zip(*text_length_metrics)
    
    # Adding prompt-based features
    df['prompt_length'] = df['Prompt'].apply(compute_prompt_features)
    
    # Classification feature: 1 for AI-generated (non-human sources) and 0 for human
    df['classification'] = df['source'].apply(lambda x: 0 if x == 'Human' else 1)
    
    # Extract embeddings and perform dimensionality reduction
    embeddings = np.array([extract_nlp_embeddings(text, model) for text in df['text']])
    pca = PCA(n_components=50)  # Adjust based on your dataset and model
    reduced_embeddings = pca.fit_transform(embeddings)
    
    # Append the reduced embeddings to the DataFrame
    for i in range(reduced_embeddings.shape[1]):
        df[f'embedding_{i}'] = reduced_embeddings[:, i]
    
    return df
df = df_merged

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
df = add_features_to_df(df, model)

with open('mlp_pca.pkl', 'wb') as file:
    pickle.dump(pca, file)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abishekjoshuat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: 

In [25]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from textstat.textstat import textstatistics
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec, KeyedVectors
import nltk
import pickle

# Ensure nltk resources are downloaded (needed for tokenization)
nltk.download('punkt')

def lexical_diversity(text):
    tokens = word_tokenize(text)
    return len(set(tokens)) / len(tokens) if tokens else 0

def average_sentence_length(text):
    sentences = sent_tokenize(text)
    tokens = word_tokenize(text)
    return len(tokens) / len(sentences) if sentences else 0

def compute_readability_score(text):
    return textstatistics().flesch_reading_ease(text)

def compute_text_length_metrics(text):
    chars = len(text)
    words = len(word_tokenize(text))
    sentences = len(sent_tokenize(text))
    return chars, words, sentences

def compute_prompt_features(prompt):
    prompt_length = len(prompt)
    return prompt_length

def extract_nlp_embeddings(text, model):
    tokens = word_tokenize(text.lower())
    tokens_in_model = [word for word in tokens if word in model.key_to_index]
    if not tokens_in_model:
        return np.zeros(model.vector_size)
    embeddings = np.mean([model.get_vector(word) for word in tokens_in_model], axis=0)
    return embeddings

def add_features_to_df(df, model, pca):
    df['lexical_diversity'] = df['text'].apply(lexical_diversity)
    df['avg_sentence_length'] = df['text'].apply(average_sentence_length)
    df['readability_score'] = df['text'].apply(compute_readability_score)
    
    text_length_metrics = df['text'].apply(compute_text_length_metrics)
    df['char_count'], df['word_count'], df['sentence_count'] = zip(*text_length_metrics)
    
    df['prompt_length'] = df['Prompt'].apply(compute_prompt_features)
    
    df['classification'] = df['source'].apply(lambda x: 0 if x == 'Human' else 1)
    
    embeddings = np.array([extract_nlp_embeddings(text, model) for text in df['text']])
    reduced_embeddings = pca.transform(embeddings)
    
    for i in range(reduced_embeddings.shape[1]):
        df[f'embedding_{i}'] = reduced_embeddings[:, i]
    
    return df

# Load the Word2Vec model
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# Assuming df is your DataFrame containing the text data
# Extract embeddings for PCA fitting
embeddings = np.array([extract_nlp_embeddings(text, model) for text in df['text']])

# Fit PCA on the embeddings
pca = PCA(n_components=50)
pca.fit(embeddings)

# Save the fitted PCA model
with open('mlp_pca.pkl', 'wb') as file:
    pickle.dump(pca, file)

# Now, transform your dataset
df_transformed = add_features_to_df(df, model, pca)

# Loading the pre-fitted PCA model
with open('mlp_pca.pkl', 'rb') as file:
    pca = pickle.load(file)

# Transform new data
# Assume new_df is the new DataFrame you want to process
new_df_transformed = add_features_to_df(df, model, pca)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abishekjoshuat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
df.source.value_counts()

Bloom-7B    100
Name: source, dtype: int64

In [21]:
df.head()

,text,source,text_length,word_count,Prompt,lexical_diversity,avg_sentence_length,readability_score,char_count,sentence_count,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,967,181,Undefined,0.806630,25.857143,57.30,967,7,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,Miles feels restless after working all day. He...,Bloom-7B,5068,924,Undefined,0.661255,23.100000,53.21,5068,40,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,So first of I am danish. That means that I fol...,Bloom-7B,1602,316,Undefined,0.718354,22.571429,61.97,1602,14,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,In this paper we present a novel rule-based ap...,Bloom-7B,5469,1015,Undefined,0.564532,40.600000,27.86,5469,25,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,"Most social progressives, love democracy, and ...",Bloom-7B,2379,437,Undefined,0.752860,23.000000,61.67,2379,19,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097


In [22]:
df.to_csv('corpus.csv')

In [2]:
import pandas as pd
frame = pd.read_csv('data.csv')
frame

,text,source,prompt_id,text_length,word_count
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,0,967,157
1,Miles feels restless after working all day. He...,Bloom-7B,0,5068,778
2,So first of I am danish. That means that I fol...,Bloom-7B,0,1602,267
3,In this paper we present a novel rule-based ap...,Bloom-7B,0,5469,848
4,"Most social progressives, love democracy, and ...",Bloom-7B,0,2379,380
...,...,...,...,...,...
788917,"\nIn the vast expanse of time, where the echoe...",YI-34B,1293,5523,870
788918,"\nThe phenomenon of brain drain, particularly ...",YI-34B,1154,4540,677
788919,\nThe Influence of Climate Change on Marine Ec...,YI-34B,2783,3889,598
788920,\nTitle: The Case for Limiting Car Usage: Navi...,YI-34B,41,3560,533


In [22]:
prompts_df = pd.read_csv('prompts.csv')
data_df = pd.read_csv('data.csv')
# Merging the DataFrames on 'prompt_id'
df_merged = pd.merge(data_df, prompts_df, left_on='prompt_id', right_on='Prompt ID', how='left')

# Optionally, if you don't need the 'Prompt ID' column anymore, you can drop it
df_merged.drop('prompt_id', axis=1, inplace=True)
df_merged.drop('Prompt ID', axis=1, inplace=True)
#df_merged.head()

In [23]:
df = df_merged
df.shape

(788922, 5)

In [24]:
df

,text,source,text_length,word_count,Prompt
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,967,157,Undefined
1,Miles feels restless after working all day. He...,Bloom-7B,5068,778,Undefined
2,So first of I am danish. That means that I fol...,Bloom-7B,1602,267,Undefined
3,In this paper we present a novel rule-based ap...,Bloom-7B,5469,848,Undefined
4,"Most social progressives, love democracy, and ...",Bloom-7B,2379,380,Undefined
...,...,...,...,...,...
788917,"\nIn the vast expanse of time, where the echoe...",YI-34B,5523,870,The Ethics of De-extinction and Bringing Back ...
788918,"\nThe phenomenon of brain drain, particularly ...",YI-34B,4540,677,The Economic and Social Consequences of Brain ...
788919,\nThe Influence of Climate Change on Marine Ec...,YI-34B,3889,598,The influence of climate change on marine ecos...
788920,\nTitle: The Case for Limiting Car Usage: Navi...,YI-34B,3560,533,Write an explanatory essay to inform fellow ci...


In [2]:
import pandas as pd
final_df = pd.read_csv('corpus.csv')
final_df.head()

,Unnamed: 0,text,source,text_length,word_count,Prompt,lexical_diversity,avg_sentence_length,readability_score,char_count,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,0,"Federal law supersedes state law, and cannabis...",Bloom-7B,967,181,Undefined,0.806630,25.857143,57.30,967,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,1,Miles feels restless after working all day. He...,Bloom-7B,5068,924,Undefined,0.661255,23.100000,53.21,5068,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,2,So first of I am danish. That means that I fol...,Bloom-7B,1602,316,Undefined,0.718354,22.571429,61.97,1602,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,3,In this paper we present a novel rule-based ap...,Bloom-7B,5469,1015,Undefined,0.564532,40.600000,27.86,5469,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,4,"Most social progressives, love democracy, and ...",Bloom-7B,2379,437,Undefined,0.752860,23.000000,61.67,2379,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097


In [2]:
pd.crosstab(final_df.readability_score,final_df.classification)

classification,0,1
readability_score,,
-2584.12,0,1
-1254.80,1,0
-1213.51,0,1
-1005.10,1,0
-973.64,0,1
...,...,...
118.78,0,1
118.89,0,4
118.99,0,1


In [3]:
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
final_df = final_df.sample(frac=1).reset_index(drop=True)
X = final_df.drop(['text','source', 'Prompt', 'classification'], axis=1)
y = final_df['classification']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# model = RandomForestClassifier(n_estimators=100)
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)
# print(f"Accuracy: {accuracy_score(y_test, predictions)}")
# print(f"Precision: {precision_score(y_test, predictions)}")
# print(f"Recall: {recall_score(y_test, predictions)}")
# print(f"F1 Score: {f1_score(y_test, predictions)}")

In [6]:
import pickle
pickle.dump(model, open('first_trained_rand_forest_model.pkl', 'wb'))

In [7]:
loaded_model = pickle.load(open('first_trained_rand_forest_model.pkl', 'rb'))
result = loaded_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, result)}")
print(f"Precision: {precision_score(y_test, result)}")
print(f"Recall: {recall_score(y_test, result)}")
print(f"F1 Score: {f1_score(y_test, result)}")

Accuracy: 0.9999873245238774
Precision: 0.9999773324870795
Recall: 1.0
F1 Score: 0.9999886661150843


In [9]:
from sklearn.model_selection import cross_val_score
import numpy as np
cv_model = RandomForestClassifier(n_estimators = 100)
num_folds = 5
scoring_metric = 'accuracy'  
scores = cross_val_score(cv_model, X, y, cv=num_folds, scoring=scoring_metric)
mean_score = np.mean(scores)
std_dev_score = np.std(scores)

print(f"Mean {scoring_metric.capitalize()} (across {num_folds} folds): {mean_score:.4f}")
print(f"Standard Deviation in {scoring_metric.capitalize()}: {std_dev_score:.4f}")


Mean Accuracy (across 5 folds): 1.0000
Standard Deviation in Accuracy: 0.0000


In [4]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize the SGD Classifier
# You can adjust the loss function, penalty, and other hyperparameters as needed
sgd_model = SGDClassifier(loss='hinge', penalty='l2', random_state=42, max_iter=1000, tol=1e-3)

# Train the model
sgd_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = sgd_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Accuracy: 0.5983
Precision: 0.5889
Recall: 0.9340
F1 Score: 0.7224


In [10]:
from sklearn.model_selection import cross_val_score
import numpy as np
cv_sdg_model = SGDClassifier(loss='hinge', penalty='l2', random_state=42, max_iter=5000,tol=1e-3)
num_folds = 5
scoring_metric = 'accuracy'  
scores = cross_val_score(cv_sdg_model, X, y, cv=num_folds, scoring=scoring_metric)
mean_score = np.mean(scores)
std_dev_score = np.std(scores)

print(f"Mean {scoring_metric.capitalize()} (across {num_folds} folds): {mean_score:.4f}")
print(f"Standard Deviation in {scoring_metric.capitalize()}: {std_dev_score:.4f}")


Mean Accuracy (across 5 folds): 0.5831
Standard Deviation in Accuracy: 0.0553


In [11]:
import pickle
pickle.dump(sgd_model, open('first_trained_sgd_model.pkl', 'wb'))

In [20]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'loss': ['hinge', 'log', 'modified_huber'],  # Different loss functions
    'penalty': ['l2', 'l1', 'elasticnet'],  # Different penalties for regularization
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Regularization strength
    'max_iter': [1000],  # Number of epochs; adjust as necessary
    'tol': [1e-3],  # Tolerance for stopping criteria
    'random_state': [42]  # Ensure reproducibility
}


In [22]:
grid_search = GridSearchCV(estimator=sgd_clf, param_grid=param_grid, 
                           n_jobs=-1, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iter

KeyboardInterrupt: 

In [12]:
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)
best_model = grid_search.best_estimator_

Best parameters: {'alpha': 0.0001, 'loss': 'hinge', 'max_iter': 1000, 'penalty': 'l1', 'random_state': 42, 'tol': 0.001}
Best cross-validation accuracy: 0.6938984618777874


In [4]:
from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression()
log_clf.fit(X_train,y_train)


LogisticRegression()

In [ ]:
from sklearn.metrics import classification_report
y_pred = log_clf.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

In [10]:
from sklearn.model_selection import GridSearchCV
log_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['saga'] 
}
log_grid_search = GridSearchCV(estimator=log_clf, param_grid=log_param_grid, n_jobs=-1, cv=5, scoring='accuracy', verbose=2)
log_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/sit

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/abishekjosh

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: Convergence

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['saga']},
             scoring='accuracy', verbose=2)

In [11]:
print("Best parameters found:", log_grid_search.best_params_)
print("Best cross-validation score:", log_grid_search.best_score_)
best_model = log_grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test set score:", test_score)

Best parameters found: {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Best cross-validation score: 0.6316679289887718
Test set score: 0.6302310105523339


In [12]:
y_pred = best_model.predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.42      0.62      0.50     47567
           1       0.79      0.63      0.71    110218

    accuracy                           0.63    157785
   macro avg       0.61      0.63      0.60    157785
weighted avg       0.68      0.63      0.64    157785



In [16]:
import pickle as pkl
with open('first_trained_sgd_model.pkl', 'rb') as file:
    sgd_clf = pkl.load(file)

In [18]:
y_pred = sgd_clf.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.5971543556104826
              precision    recall  f1-score   support

           0       0.17      0.67      0.27     17730
           1       0.93      0.59      0.72    140055

    accuracy                           0.60    157785
   macro avg       0.55      0.63      0.50    157785
weighted avg       0.85      0.60      0.67    157785



In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train, y_train)
support_vectors = svm.support_vectors_

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('corpus.csv')
df.head()

,Unnamed: 0,text,source,text_length,word_count,Prompt,lexical_diversity,avg_sentence_length,readability_score,char_count,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,0,"Federal law supersedes state law, and cannabis...",Bloom-7B,967,181,Undefined,0.806630,25.857143,57.30,967,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,1,Miles feels restless after working all day. He...,Bloom-7B,5068,924,Undefined,0.661255,23.100000,53.21,5068,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,2,So first of I am danish. That means that I fol...,Bloom-7B,1602,316,Undefined,0.718354,22.571429,61.97,1602,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,3,In this paper we present a novel rule-based ap...,Bloom-7B,5469,1015,Undefined,0.564532,40.600000,27.86,5469,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,4,"Most social progressives, love democracy, and ...",Bloom-7B,2379,437,Undefined,0.752860,23.000000,61.67,2379,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097


In [2]:
df.drop(['text_length','word_count'],axis=1,inplace=True)

In [3]:
from sklearn.model_selection import train_test_split
df.sample(frac=1).reset_index(drop=True)
X = df.drop(['text','source','Prompt','prompt_length','avg_sentence_length','char_count','sentence_count','classification'],axis=1)
y = df['classification']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [7]:
df.classification.value_counts()

1    441230
0    347692
Name: classification, dtype: int64

In [8]:
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import LabelEncoder
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

In [14]:
y_resampled_decoded = label_encoder.inverse_transform(y_resampled)

0    353014
1    353014
Name: classification, dtype: int64

In [32]:
from sklearn.linear_model import LogisticRegression
log_reg_clf = LogisticRegression()
log_reg_clf.fit(X_resampled,y_resampled)

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [34]:
X_test.drop('Unnamed: 0',axis=1,inplace=True)

In [36]:
from sklearn.metrics import accuracy_score, classification_report
log_reg_y_pred = log_reg_clf.predict(X_test)
print('Accuracy:',accuracy_score(log_reg_y_pred,y_test))
print(classification_report(log_reg_y_pred,y_test))

Accuracy: 0.723883765883956
              precision    recall  f1-score   support

           0       0.72      0.67      0.70     74536
           1       0.72      0.77      0.75     83249

    accuracy                           0.72    157785
   macro avg       0.72      0.72      0.72    157785
weighted avg       0.72      0.72      0.72    157785



In [37]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier()
sgd_clf.fit(X_resampled,y_resampled)

SGDClassifier()

In [38]:
from sklearn.metrics import accuracy_score, classification_report
sgd_clf_y_pred = sgd_clf.predict(X_test)
print('Accuracy:',accuracy_score(sgd_clf_y_pred,y_test))
print(classification_report(sgd_clf_y_pred,y_test))

Accuracy: 0.6939189403301962
              precision    recall  f1-score   support

           0       0.48      0.73      0.58     45460
           1       0.86      0.68      0.76    112325

    accuracy                           0.69    157785
   macro avg       0.67      0.71      0.67    157785
weighted avg       0.75      0.69      0.71    157785



In [39]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_resampled, y_resampled)

MLPClassifier()

In [40]:
mlp_y_pred = mlp.predict(X_test)
print('Accuracy:',accuracy_score(mlp_y_pred,y_test))
print(classification_report(mlp_y_pred,y_test))

Accuracy: 0.8633076654941851
              precision    recall  f1-score   support

           0       0.81      0.87      0.84     64701
           1       0.91      0.86      0.88     93084

    accuracy                           0.86    157785
   macro avg       0.86      0.86      0.86    157785
weighted avg       0.87      0.86      0.86    157785



In [41]:
from sklearn.model_selection import GridSearchCV
log_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['saga'] 
}
log_grid_search = GridSearchCV(estimator=log_reg_clf, param_grid=log_param_grid, n_jobs=-1, cv=5, scoring='accuracy', verbose=2)
log_grid_search.fit(X_resampled, y_resampled)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/sit

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: Convergence

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-

/Users/abishekjoshuat/Developer/Sample_ML_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['saga']},
             scoring='accuracy', verbose=2)

[CV] END ...................C=0.001, penalty=l2, solver=saga; total time= 1.8min
[CV] END ...................C=0.001, penalty=l2, solver=saga; total time= 1.7min
[CV] END ....................C=0.01, penalty=l1, solver=saga; total time= 1.8min
[CV] END ..................C=0.01, penalty=none, solver=saga; total time= 1.5min
[CV] END .....................C=0.1, penalty=l2, solver=saga; total time= 1.8min
[CV] END .............C=0.1, penalty=elasticnet, solver=saga; total time=   0.7s
[CV] END .............C=0.1, penalty=elasticnet, solver=saga; total time=   0.4s
[CV] END .............C=0.1, penalty=elasticnet, solver=saga; total time=   0.5s
[CV] END .............C=0.1, penalty=elasticnet, solver=saga; total time=   0.4s
[CV] END .............C=0.1, penalty=elasticnet, solver=saga; total time=   0.4s
[CV] END ...................C=0.1, penalty=none, solver=saga; total time= 1.6min
[CV] END .......................C=1, penalty=l1, solver=saga; total time= 1.7min
[CV] END ...................

[CV] END ...................C=0.001, penalty=l1, solver=saga; total time= 2.0min
[CV] END .................C=0.001, penalty=none, solver=saga; total time= 1.7min
[CV] END ....................C=0.01, penalty=l2, solver=saga; total time= 1.5min
[CV] END ............C=0.01, penalty=elasticnet, solver=saga; total time=   0.5s
[CV] END ............C=0.01, penalty=elasticnet, solver=saga; total time=   0.5s
[CV] END ..................C=0.01, penalty=none, solver=saga; total time= 1.6min
[CV] END .....................C=0.1, penalty=l1, solver=saga; total time= 2.0min
[CV] END ...................C=0.1, penalty=none, solver=saga; total time= 1.6min
[CV] END .......................C=1, penalty=l2, solver=saga; total time= 1.5min
[CV] END .....................C=1, penalty=none, solver=saga; total time= 1.5min
[CV] END ......................C=10, penalty=l2, solver=saga; total time= 1.6min
[CV] END ....................C=10, penalty=none, solver=saga; total time= 1.5min
[CV] END ...................

In [42]:
print("Best parameters found:", log_grid_search.best_params_)
#print("Best cross-validation score:", log_grid_search.best_score_)
log_reg_best_model = log_grid_search.best_estimator_
log_reg_best_y_pred = log_reg_best_model.predict(X_test)
print('Accuracy:',accuracy_score(log_reg_best_y_pred,y_test))
print(classification_report(log_reg_best_y_pred,y_test))

Best parameters found: {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Accuracy: 0.6671736857115695
              precision    recall  f1-score   support

           0       0.68      0.61      0.64     77312
           1       0.66      0.72      0.69     80473

    accuracy                           0.67    157785
   macro avg       0.67      0.67      0.67    157785
weighted avg       0.67      0.67      0.67    157785



In [43]:
import pickle
pickle.dump(mlp, open('mlp.pkl', 'wb'))

In [44]:
X_resampled

,lexical_diversity,readability_score,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,0.300033,18.990000,-0.248195,-0.179635,-0.083045,0.010855,0.105388,0.105773,-0.079635,0.003656,...,0.023005,-0.016056,0.010822,-0.000577,0.020482,0.000518,-0.007269,0.005182,-0.002655,-0.006325
1,0.451613,80.820000,0.231140,0.214960,-0.056012,-0.057660,0.021940,0.025757,-0.012500,-0.041915,...,0.007638,0.022504,-0.005386,-0.012064,0.011904,-0.018995,-0.023025,0.009447,0.014939,-0.000303
2,0.617284,50.330000,0.025687,-0.154967,0.035850,-0.040272,0.104272,-0.110137,-0.092216,-0.008114,...,0.001428,0.002351,0.011762,0.025877,-0.010867,-0.044785,-0.003382,0.075150,-0.005372,0.008051
3,0.372159,37.910000,-0.199305,0.011802,-0.067226,0.116867,0.031592,0.039090,-0.006330,-0.003103,...,0.021811,-0.007376,0.005410,-0.005284,-0.004893,0.004136,0.026057,0.014127,-0.006860,0.013506
4,0.611940,38.250000,-0.145215,0.034901,-0.022185,-0.129232,0.024889,-0.006497,0.037708,0.038051,...,0.020041,0.014329,-0.025540,-0.030381,-0.037740,-0.022946,-0.014160,-0.014366,-0.005830,-0.026705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706023,0.499839,54.745171,0.082951,0.058508,-0.049192,0.108803,0.103551,0.000312,0.042249,0.014440,...,-0.026842,0.004354,-0.000510,-0.019718,0.002946,-0.029177,-0.017638,-0.031907,-0.008844,0.020305
706024,0.541621,59.655854,0.075304,0.152936,-0.028156,0.059426,0.075589,-0.055549,0.016988,-0.000330,...,-0.013487,-0.007866,0.030421,-0.012009,0.014661,-0.028032,-0.019619,-0.002625,-0.018203,0.009254
706025,0.502306,54.355693,0.078595,0.045208,-0.045423,0.101048,0.127583,-0.006447,0.050258,0.026254,...,-0.019883,0.008003,0.009772,-0.022577,0.006393,-0.029919,-0.027532,-0.032977,-0.013356,0.015409
706026,0.508887,54.631433,0.091584,0.048740,-0.046034,0.109513,0.128275,-0.003456,0.053254,0.027267,...,-0.022960,0.006409,0.008017,-0.024181,0.006599,-0.030657,-0.026145,-0.034481,-0.012717,0.016915


In [2]:
import pandas as pd
df = pd.read_csv('corpus.csv')
#df.source.unique()
df.head()

,Unnamed: 0,text,source,text_length,word_count,Prompt,lexical_diversity,avg_sentence_length,readability_score,char_count,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,0,"Federal law supersedes state law, and cannabis...",Bloom-7B,967,181,Undefined,0.806630,25.857143,57.30,967,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,1,Miles feels restless after working all day. He...,Bloom-7B,5068,924,Undefined,0.661255,23.100000,53.21,5068,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,2,So first of I am danish. That means that I fol...,Bloom-7B,1602,316,Undefined,0.718354,22.571429,61.97,1602,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,3,In this paper we present a novel rule-based ap...,Bloom-7B,5469,1015,Undefined,0.564532,40.600000,27.86,5469,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,4,"Most social progressives, love democracy, and ...",Bloom-7B,2379,437,Undefined,0.752860,23.000000,61.67,2379,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097


In [3]:
df.shape

(788922, 63)

In [4]:
df.drop(['Unnamed: 0'], axis=1, inplace = True)
df

,text,source,text_length,word_count,Prompt,lexical_diversity,avg_sentence_length,readability_score,char_count,sentence_count,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,967,181,Undefined,0.806630,25.857143,57.30,967,7,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,Miles feels restless after working all day. He...,Bloom-7B,5068,924,Undefined,0.661255,23.100000,53.21,5068,40,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,So first of I am danish. That means that I fol...,Bloom-7B,1602,316,Undefined,0.718354,22.571429,61.97,1602,14,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,In this paper we present a novel rule-based ap...,Bloom-7B,5469,1015,Undefined,0.564532,40.600000,27.86,5469,25,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,"Most social progressives, love democracy, and ...",Bloom-7B,2379,437,Undefined,0.752860,23.000000,61.67,2379,19,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788917,"\nIn the vast expanse of time, where the echoe...",YI-34B,5523,951,The Ethics of De-extinction and Bringing Back ...,0.420610,28.818182,36.22,5523,33,...,-0.005222,-0.001365,-0.020148,-0.023085,-0.030199,-0.012152,0.002733,-0.000128,0.022612,-0.007202
788918,"\nThe phenomenon of brain drain, particularly ...",YI-34B,4540,761,The Economic and Social Consequences of Brain ...,0.395532,28.185185,29.08,4540,27,...,-0.006157,-0.023404,0.037557,0.026361,-0.008231,0.023823,-0.009165,-0.004096,-0.016606,0.020802
788919,\nThe Influence of Climate Change on Marine Ec...,YI-34B,3889,670,The influence of climate change on marine ecos...,0.447761,26.800000,47.22,3889,25,...,0.010574,-0.008166,0.018903,-0.001044,-0.037097,0.025732,0.039127,0.018915,0.072912,-0.009205
788920,\nTitle: The Case for Limiting Car Usage: Navi...,YI-34B,3560,592,Write an explanatory essay to inform fellow ci...,0.476351,28.190476,28.77,3560,21,...,0.013639,0.035284,0.000662,0.023736,0.030208,0.013736,-0.011754,-0.010964,0.000265,0.014060


In [5]:
df.drop(['text_length', 'word_count', 'Prompt', 'avg_sentence_length', 'char_count', 'sentence_count'], axis=1, inplace = True)
df

,text,source,lexical_diversity,readability_score,prompt_length,classification,embedding_0,embedding_1,embedding_2,embedding_3,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,0.806630,57.30,9,1,0.008560,0.095722,-0.049095,-0.118440,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,Miles feels restless after working all day. He...,Bloom-7B,0.661255,53.21,9,1,0.070761,-0.099866,0.107777,0.008168,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,So first of I am danish. That means that I fol...,Bloom-7B,0.718354,61.97,9,1,0.164173,0.117847,-0.012992,0.088472,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,In this paper we present a novel rule-based ap...,Bloom-7B,0.564532,27.86,9,1,-0.179447,0.010308,0.155961,0.029293,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,"Most social progressives, love democracy, and ...",Bloom-7B,0.752860,61.67,9,1,0.159706,0.083560,-0.074277,0.031704,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788917,"\nIn the vast expanse of time, where the echoe...",YI-34B,0.420610,36.22,61,1,-0.157033,-0.008564,-0.079678,0.104863,...,-0.005222,-0.001365,-0.020148,-0.023085,-0.030199,-0.012152,0.002733,-0.000128,0.022612,-0.007202
788918,"\nThe phenomenon of brain drain, particularly ...",YI-34B,0.395532,29.08,66,1,-0.282313,0.034885,-0.078380,-0.014602,...,-0.006157,-0.023404,0.037557,0.026361,-0.008231,0.023823,-0.009165,-0.004096,-0.016606,0.020802
788919,\nThe Influence of Climate Change on Marine Ec...,YI-34B,0.447761,47.22,52,1,-0.264905,-0.011840,0.106972,0.034591,...,0.010574,-0.008166,0.018903,-0.001044,-0.037097,0.025732,0.039127,0.018915,0.072912,-0.009205
788920,\nTitle: The Case for Limiting Car Usage: Navi...,YI-34B,0.476351,28.77,512,1,-0.263766,0.064379,-0.021544,-0.112702,...,0.013639,0.035284,0.000662,0.023736,0.030208,0.013736,-0.011754,-0.010964,0.000265,0.014060


In [6]:
df.drop(['source'], axis=1, inplace = True)
df

,text,lexical_diversity,readability_score,prompt_length,classification,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,"Federal law supersedes state law, and cannabis...",0.806630,57.30,9,1,0.008560,0.095722,-0.049095,-0.118440,-0.004019,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,Miles feels restless after working all day. He...,0.661255,53.21,9,1,0.070761,-0.099866,0.107777,0.008168,-0.063522,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,So first of I am danish. That means that I fol...,0.718354,61.97,9,1,0.164173,0.117847,-0.012992,0.088472,0.097658,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,In this paper we present a novel rule-based ap...,0.564532,27.86,9,1,-0.179447,0.010308,0.155961,0.029293,0.099195,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,"Most social progressives, love democracy, and ...",0.752860,61.67,9,1,0.159706,0.083560,-0.074277,0.031704,0.066774,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788917,"\nIn the vast expanse of time, where the echoe...",0.420610,36.22,61,1,-0.157033,-0.008564,-0.079678,0.104863,-0.095362,...,-0.005222,-0.001365,-0.020148,-0.023085,-0.030199,-0.012152,0.002733,-0.000128,0.022612,-0.007202
788918,"\nThe phenomenon of brain drain, particularly ...",0.395532,29.08,66,1,-0.282313,0.034885,-0.078380,-0.014602,-0.034506,...,-0.006157,-0.023404,0.037557,0.026361,-0.008231,0.023823,-0.009165,-0.004096,-0.016606,0.020802
788919,\nThe Influence of Climate Change on Marine Ec...,0.447761,47.22,52,1,-0.264905,-0.011840,0.106972,0.034591,-0.192508,...,0.010574,-0.008166,0.018903,-0.001044,-0.037097,0.025732,0.039127,0.018915,0.072912,-0.009205
788920,\nTitle: The Case for Limiting Car Usage: Navi...,0.476351,28.77,512,1,-0.263766,0.064379,-0.021544,-0.112702,-0.044006,...,0.013639,0.035284,0.000662,0.023736,0.030208,0.013736,-0.011754,-0.010964,0.000265,0.014060


In [7]:
df.drop(['prompt_length'], axis=1, inplace = True)
df

,text,lexical_diversity,readability_score,classification,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,"Federal law supersedes state law, and cannabis...",0.806630,57.30,1,0.008560,0.095722,-0.049095,-0.118440,-0.004019,0.043696,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,Miles feels restless after working all day. He...,0.661255,53.21,1,0.070761,-0.099866,0.107777,0.008168,-0.063522,0.097034,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,So first of I am danish. That means that I fol...,0.718354,61.97,1,0.164173,0.117847,-0.012992,0.088472,0.097658,-0.002207,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,In this paper we present a novel rule-based ap...,0.564532,27.86,1,-0.179447,0.010308,0.155961,0.029293,0.099195,-0.101426,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,"Most social progressives, love democracy, and ...",0.752860,61.67,1,0.159706,0.083560,-0.074277,0.031704,0.066774,0.062544,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788917,"\nIn the vast expanse of time, where the echoe...",0.420610,36.22,1,-0.157033,-0.008564,-0.079678,0.104863,-0.095362,0.039880,...,-0.005222,-0.001365,-0.020148,-0.023085,-0.030199,-0.012152,0.002733,-0.000128,0.022612,-0.007202
788918,"\nThe phenomenon of brain drain, particularly ...",0.395532,29.08,1,-0.282313,0.034885,-0.078380,-0.014602,-0.034506,0.077010,...,-0.006157,-0.023404,0.037557,0.026361,-0.008231,0.023823,-0.009165,-0.004096,-0.016606,0.020802
788919,\nThe Influence of Climate Change on Marine Ec...,0.447761,47.22,1,-0.264905,-0.011840,0.106972,0.034591,-0.192508,0.167850,...,0.010574,-0.008166,0.018903,-0.001044,-0.037097,0.025732,0.039127,0.018915,0.072912,-0.009205
788920,\nTitle: The Case for Limiting Car Usage: Navi...,0.476351,28.77,1,-0.263766,0.064379,-0.021544,-0.112702,-0.044006,0.123673,...,0.013639,0.035284,0.000662,0.023736,0.030208,0.013736,-0.011754,-0.010964,0.000265,0.014060


In [8]:
df.drop(['text'], axis=1, inplace = True)
df

,lexical_diversity,readability_score,classification,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,0.806630,57.30,1,0.008560,0.095722,-0.049095,-0.118440,-0.004019,0.043696,-0.116908,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,0.661255,53.21,1,0.070761,-0.099866,0.107777,0.008168,-0.063522,0.097034,0.091039,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,0.718354,61.97,1,0.164173,0.117847,-0.012992,0.088472,0.097658,-0.002207,-0.073280,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,0.564532,27.86,1,-0.179447,0.010308,0.155961,0.029293,0.099195,-0.101426,-0.038001,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,0.752860,61.67,1,0.159706,0.083560,-0.074277,0.031704,0.066774,0.062544,-0.055907,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788917,0.420610,36.22,1,-0.157033,-0.008564,-0.079678,0.104863,-0.095362,0.039880,-0.053883,...,-0.005222,-0.001365,-0.020148,-0.023085,-0.030199,-0.012152,0.002733,-0.000128,0.022612,-0.007202
788918,0.395532,29.08,1,-0.282313,0.034885,-0.078380,-0.014602,-0.034506,0.077010,0.023160,...,-0.006157,-0.023404,0.037557,0.026361,-0.008231,0.023823,-0.009165,-0.004096,-0.016606,0.020802
788919,0.447761,47.22,1,-0.264905,-0.011840,0.106972,0.034591,-0.192508,0.167850,-0.146308,...,0.010574,-0.008166,0.018903,-0.001044,-0.037097,0.025732,0.039127,0.018915,0.072912,-0.009205
788920,0.476351,28.77,1,-0.263766,0.064379,-0.021544,-0.112702,-0.044006,0.123673,0.041577,...,0.013639,0.035284,0.000662,0.023736,0.030208,0.013736,-0.011754,-0.010964,0.000265,0.014060


In [9]:
df['classification'].value_counts()

1    441230
0    347692
Name: classification, dtype: int64

In [10]:
X = df.drop(['classification'], axis=1)
X

,lexical_diversity,readability_score,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,0.806630,57.30,0.008560,0.095722,-0.049095,-0.118440,-0.004019,0.043696,-0.116908,-0.008107,...,-0.035092,-0.001925,-0.041666,-0.005266,0.034543,0.002532,-0.071304,0.014727,-0.012865,0.004744
1,0.661255,53.21,0.070761,-0.099866,0.107777,0.008168,-0.063522,0.097034,0.091039,0.015570,...,-0.003091,-0.043269,0.003721,-0.005574,0.030584,-0.006616,0.030465,0.000941,0.020913,0.060853
2,0.718354,61.97,0.164173,0.117847,-0.012992,0.088472,0.097658,-0.002207,-0.073280,-0.042987,...,0.011013,-0.008409,0.008066,-0.012316,0.007549,-0.000401,-0.001133,-0.005499,-0.011693,0.000179
3,0.564532,27.86,-0.179447,0.010308,0.155961,0.029293,0.099195,-0.101426,-0.038001,0.011437,...,-0.018165,0.012030,-0.020475,-0.006033,0.002823,-0.000594,0.000835,0.001109,0.009936,-0.019899
4,0.752860,61.67,0.159706,0.083560,-0.074277,0.031704,0.066774,0.062544,-0.055907,-0.002149,...,0.004131,0.014534,0.007382,-0.016189,0.007070,0.001430,-0.017871,0.004868,0.010563,0.008097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788917,0.420610,36.22,-0.157033,-0.008564,-0.079678,0.104863,-0.095362,0.039880,-0.053883,-0.106344,...,-0.005222,-0.001365,-0.020148,-0.023085,-0.030199,-0.012152,0.002733,-0.000128,0.022612,-0.007202
788918,0.395532,29.08,-0.282313,0.034885,-0.078380,-0.014602,-0.034506,0.077010,0.023160,0.014074,...,-0.006157,-0.023404,0.037557,0.026361,-0.008231,0.023823,-0.009165,-0.004096,-0.016606,0.020802
788919,0.447761,47.22,-0.264905,-0.011840,0.106972,0.034591,-0.192508,0.167850,-0.146308,-0.042047,...,0.010574,-0.008166,0.018903,-0.001044,-0.037097,0.025732,0.039127,0.018915,0.072912,-0.009205
788920,0.476351,28.77,-0.263766,0.064379,-0.021544,-0.112702,-0.044006,0.123673,0.041577,-0.016348,...,0.013639,0.035284,0.000662,0.023736,0.030208,0.013736,-0.011754,-0.010964,0.000265,0.014060


In [11]:
y = df['classification']
y

0         1
1         1
2         1
3         1
4         1
         ..
788917    1
788918    1
788919    1
788920    1
788921    1
Name: classification, Length: 788922, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2) 

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(631137, 52)
(157785, 52)
(631137,)
(157785,)


In [14]:
from imblearn.over_sampling import ADASYN
adasyn = ADASYN()
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

In [15]:
X_resampled

,lexical_diversity,readability_score,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49
0,0.362340,19.810000,-0.313918,0.000198,-0.037702,0.178112,-0.057451,0.010045,0.070563,-0.030519,...,-0.031118,-0.018364,-0.017552,-0.033377,0.010988,0.024157,0.016149,0.042856,0.000855,-0.005194
1,0.594340,72.460000,-0.028841,-0.119339,0.142218,-0.050424,0.123454,-0.044138,0.121926,-0.055619,...,-0.023218,0.030872,0.012089,0.004924,0.044491,0.011058,0.017052,-0.009379,0.006174,-0.012802
2,0.577206,69.920000,0.303838,-0.030253,-0.017719,-0.050776,-0.013972,-0.028626,0.036949,0.004492,...,-0.019381,-0.002734,0.002830,-0.002202,0.003834,0.012343,0.003326,-0.019975,0.016681,-0.009854
3,0.397780,37.300000,-0.249733,-0.004098,0.011451,0.131737,0.081940,-0.027256,0.036708,0.097219,...,0.021817,-0.005670,-0.001689,0.042396,-0.030133,-0.009946,0.038410,0.025292,0.023069,0.017276
4,0.555921,37.640000,-0.118131,0.060025,-0.131267,0.049748,0.057296,-0.017713,-0.005860,0.073022,...,-0.017031,0.005346,0.001600,0.002799,0.008881,-0.002948,-0.008303,0.021185,0.000295,-0.030139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685751,0.865150,57.270000,0.040462,0.243818,-0.027323,-0.109279,0.062646,-0.006073,0.010005,-0.043090,...,-0.060221,-0.010429,0.068361,0.049456,-0.042433,0.030335,-0.039211,-0.005996,-0.064874,0.008871
685752,0.591880,71.572360,0.051193,0.049561,0.031480,-0.024915,-0.133134,0.047781,-0.112202,-0.018199,...,-0.005299,-0.004613,0.009455,-0.006393,-0.005214,-0.039298,-0.006475,0.007667,0.018191,0.038808
685753,0.611837,37.421336,-0.174576,0.029990,-0.020118,-0.119191,0.092546,0.062916,-0.005921,-0.010503,...,0.006864,0.004312,0.005187,0.011893,0.012100,0.012220,-0.021409,0.006099,0.011928,-0.005967
685754,0.398512,69.393325,0.064953,-0.077371,-0.005291,-0.114237,-0.021544,-0.070255,-0.048984,-0.055847,...,-0.007533,0.004589,0.017387,-0.031607,0.017932,-0.008081,0.023041,-0.008119,-0.009037,0.001978


In [16]:
y_resampled.value_counts()

1    353097
0    332659
Name: classification, dtype: int64

In [19]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)

model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_pred = model.predict(X_test)

print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred, y_test))

0.8706404284310929
              precision    recall  f1-score   support

           0       0.84      0.86      0.85     68105
           1       0.89      0.88      0.89     89680

    accuracy                           0.87    157785
   macro avg       0.87      0.87      0.87    157785
weighted avg       0.87      0.87      0.87    157785



In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()

knn_model.fit(X_resampled, y_resampled)

KNeighborsClassifier()

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_pred_knn = knn_model.predict(X_test)

print(accuracy_score(y_pred_knn,y_test))
print(classification_report(y_test, y_pred_knn))

0.7814874671229838
              precision    recall  f1-score   support

           0       0.70      0.87      0.78     69652
           1       0.87      0.71      0.78     88133

    accuracy                           0.78    157785
   macro avg       0.79      0.79      0.78    157785
weighted avg       0.80      0.78      0.78    157785



Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV 1/5] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.875 total time= 4.8min
[CV 2/5] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.875 total time= 4.7min
[CV 3/5] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.875 total time= 4.7min
[CV 4/5] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.875 total time= 4.4min
[CV 5/5] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.827 total time= 4.5min
[CV 1/5] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.880 total time= 8.7min
[CV 2/5] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.881 total ti

KeyboardInterrupt: 

In [31]:
param_grid_final = {
    'n_estimators': [150, 200], # Number of trees in the forest
    'max_depth': [None, 10, 20], # Maximum depth of the trees
    'criterion': ['gini', 'entropy', 'log_loss'],  # Function to measure the quality of a split
}

In [32]:
from sklearn.model_selection import GridSearchCV

GCVRandomForest = GridSearchCV(estimator=model, param_grid=param_grid_final, cv=5, scoring='accuracy', verbose=3)
GCVRandomForest.fit(X_resampled, y_resampled)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=150;, score=0.883 total time=13.4min
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=150;, score=0.883 total time=13.9min
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=150;, score=0.884 total time=13.9min
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=150;, score=0.883 total time=13.8min
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=150;, score=0.831 total time=14.3min
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.884 total time=18.3min
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.885 total time=32.7min
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.884 total time=19.5min
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.883 total time=23.8min
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=200;, score=0.832 total ti

KeyboardInterrupt: 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 